# A neural network to classify images of clothing

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np

We're using a dataset called 'Fashion MNIST'. GitHub: https://github.com/zalandoresearch/fashion-mnist.

This dataset contains 60,000 training images and 10,000 testing images. Each image is a 28x28px grayscale image of a fashion item, associated with a label from 10 classes. The task is to predict the class of the image (e.g. t-shirt , dress, etc.). Each image has a resolution of 28x28 pixels and is grayscale (1 channel). The grayscale values range from 0 to 255, where 0 represents black and 255 represents white.

| Label | Item |
|-------|------|
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |
<p></p>

<div style="display: flex; justify-content: center;">
    <div style="margin-right: 20px;">
        <img src="mnist.png" width=400px alt="mnist dataset image">
    </div>
    <div>
        <img src="mnist.gif" width=600px alt="mnist classes">
    </div>
</div>

## Loading data and normalization

> `tfds.load` loads the data.

> `split` specifies which splits of the dataset is to be loaded. You set as_supervised to True to ensure that the loaded tf.data.Dataset will have a 2-tuple structure (input, label).

> `ds_train` and `ds_test` are of type `tf.data.Dataset`. `ds_train` has 60,000 images which will be used for training the model. `ds_test` has 10,000 images which will be used for evaluating the model.

In [ ]:
# Define, load and configure data
(ds_train, ds_test), info = tfds.load('fashion_mnist', split=['train', 'test'], with_info=True, as_supervised=True)

In [ ]:
header = info.features['label'].num_classes
print(f"Number of classes: {header}")
print(f"Number of training examples: {info.splits['train'].num_examples}")
print(f"Number of test examples: {info.splits['test'].num_examples}")

In [ ]:
# Minium and maximum values before normalization
image_batch, labels_batch = next(iter(ds_train))
print("Before normalization ->", np.min(image_batch[0]), np.max(image_batch[0]))

> `Batch size` refers to the number of training examples utilized in one iteration.

We'll be normalizing the data to exist between 0 and 1.

code given below uses the `map()` function of `tf.data.Dataset` to apply the normalization to images in `ds_train` and `ds_test`. Since the pixel values are of type `tf.uint8`, the `tf.cast` function is used to convert them to `tf.float32` and then divide by 255.0. The dataset is also converted into batches by calling the `batch()` method with BATCH_SIZE as the argument.

In [ ]:
# Define batch size
BATCH_SIZE = 32

In [ ]:
# Normalize and batch process the dataset
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).batch(BATCH_SIZE)
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).batch(BATCH_SIZE)

In [ ]:
# Min and max values of the batch after normalization
image_batch, labels_batch = next(iter(ds_train))
print("After normalization ->", np.min(image_batch[0]), np.max(image_batch[0]))

## Design, Compile and Train

In [ ]:
# Define the model
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

0. Sequential

- This is the default mode. It means that the model will process the input data in a sequential manner, i.e., one sample at a time. This is the most common mode and is suitable for most use cases.


1. Flatten Layer

- Converts the 2D image array (28x28 pixels) into a 1D array (784 pixels)
- No parameters to learn, just reshapes the data
- Input: (28, 28) → Output: (784)

2. Dense (addds a layer of neurons) Hidden Layer
- 64 neurons in this hidden layer
- ReLU activation function (Rectified Linear Unit)
- Takes flattened input (784) and learns patterns
- Parameters: 784 * 64 + 64 = 50,240 (weights + biases)

3. Output Layer
- 10 neurons (one for each clothing class)
- Softmax activation for probability distribution
- Parameters: 64 * 10 + 10 = 650 (weights + biases)

Each layer of neurons needs an activation function to decide if a neuron should be activated or not. There are lots of options, but this lab uses the following ones.
- `Relu` effectively means if X>0 return X, else return 0. It passes values 0 or greater to the next layer in the network.
- `Softmax` takes a set of values, and effectively picks the biggest one so you don't have to sort to find the largest value. For example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it returns [0,0,0,0,1,0,0,0,0].

<div style="text-align: center;">
    <img src="network_1.svg" width="600px" alt="">
</div>


The network architecture consists of:
- Input: 28x28 grayscale image
- Flatten → Dense(64) → Dense(10)
- Total trainable parameters: 50,890

Output is a probability distribution across 10 clothing classes:
- T-shirt/top, Trouser, Pullover, Dress, Coat etc

## Compilation

The goal is for the model to figure out the relationship between the training data and its labels. Once training is complete, you want your model to see fresh images of clothing that resembles your training data and make predictions about what class of clothing they belong to.

> `Optimizer` is an algorithm that modifies the attributes of the neural network like weights and learning rate. This helps in reducing the loss and improving accuracy.

> `Loss` indicates the model's performance by a number. If the model is performing better, loss will be a smaller number. Otherwise loss will be a larger number.

> `Metrics` parameter allows TensorFlow to report on the accuracy of the training after each epoch by checking the predicted results against the known answers(labels). It basically reports back on how effectively the training is progressing. For 0.9053 means the models is 90.53% accurate.

Model.fit will train the model for a fixed number of epochs.

In [ ]:
# Compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(ds_train, epochs=5)

## Evaluating models's performance on unseen data

model.evaluate, pass in the test set, and it reports back the loss and accuracy.

In [ ]:
# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(ds_test)

print(f"\nTest accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

## Saving the model

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. We will save and load the model now. 

The above code shows how you can save the model in two different formats and load the saved model back. You can choose any format according to your use case. At the end of the output you will see two sets of model summaries. The first one shows the summary after the model is saved in the SavedModel format. The second one shows the summary after the model is saved in the h5 format.

Both model summaries are identical since we are effectively saving the same model in two different formats.

In [ ]:
# Save the entire model as a Keras model using .keras format
model.save('saved_model.keras') 

# Load the model using custom_objects to handle the custom activation function
new_model = tf.keras.models.load_model('saved_model.keras', custom_objects={'softmax_v2': tf.keras.activations.softmax})

# Summary of loaded SavedModel
new_model.summary()

# Save the entire model to a keras file.
model.save('my_model.keras')

# Recreate the exact same model, including its weights and the optimizer
new_model_keras = tf.keras.models.load_model('my_model.keras', custom_objects={'softmax_v2': tf.keras.activations.softmax})

# Summary of loaded keras model
new_model_keras.summary()